## Notes
- Get department year and name from department column
- input: clean1_bookIssue.csv
- output: clean2_bookIssue.csv

In [1]:
import re
import csv
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv("outputs/clean1_bookIssue.csv")


In [3]:
df.columns

Index(['adm_no', 'acc_no', 'title', 'issue_date', 'department'], dtype='object')

In [4]:
unq_dpt = df["department"].drop_duplicates()

In [5]:
def getYear(s):
    _ = re.search(r'[0-9]{4}', s)
    if(_ is not None):
        return _[0]
    return np.nan


def getDept(s):
    _ = re.sub(r"[0-9]{4}", "", s)
    _ = _.replace("-", " ").strip()
    if (_.find("nan") != -1):
        return np.nan
    return _


In [6]:
replace_dept = {}
for x in unq_dpt:
    if x == "NODEPT":
        replace_dept[x] = {"year": np.nan, "branch": np.nan}
        continue
    replace_dept[x] = {"year": getYear(str(x)), "branch": getDept(str(x))}
# print(replace_dept)

In [7]:
# Create two cols department and batch from dept column

depts = []
years = []

for i in df["department"]:
    try:
        depts.append(replace_dept[i]["branch"])
        years.append(replace_dept[i]["year"])
    except KeyError:
        depts.append(None)
        years.append("")

df["department"] = depts
df["batch"] = years


In [8]:
df.to_csv("outputs/clean2_bookIssueCompiled.csv", index=False)

In [9]:
# Extracting book issue details 
bk_issue = df[["adm_no", "acc_no", "title", "issue_date"]]
bk_issue.to_csv("outputs/clean2_bookIssue.csv", index=False)

In [10]:
# Extracting lib users from book issue
lib_user = df[["adm_no", "department", "batch"]]
lib_user = lib_user.rename(columns={"adm_no": "id"})
lib_user.columns
# drop duplicates

lib_user = lib_user.drop_duplicates(subset=['id'])

lib_user.to_csv("outputs/clean2_libUsersBookIssue.csv", index = False)